# 단층 퍼셉트론

In [4]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

# 커스텀 데이터세트
class CustomDataset(Dataset):
  def __init__(self, file_path):
    df = pd.read_csv(file_path)
    self.x1 = df.iloc[:, 0].values
    self.x2 = df.iloc[:, 1].values
    self.y = df.iloc[:, 2].values
    self.length= len(df)

  def __getitem__(self, index):
    x = torch.FloatTensor([self.x1[index], self.x2[index]])
    y = torch.FloatTensor([self.y[index]])
    return x, y

  def __len__(self):
    return self.length

# 커스텀 모델
class CustomModel(nn.Module):
  def __init__(self):
    super().__init__()

    self.layer = nn.Sequential(
        nn.Linear(2, 1),
        nn.Sigmoid()
    )

  def forward(self, x):
    x = self.layer(x)
    return x

In [5]:
# 데이터세트, 데이터로더
train_dataset = CustomDataset("/content/perceptron.csv")
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

In [7]:
# model, Loss, optimizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [8]:
# 모델 학습
for epoch in range(10000):
  cost = 0.0

  for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)
    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    cost += loss

  cost = cost / len(train_dataloader)

  if (epoch + 1) % 1000 == 0:
    print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

/tmp/ipython-input-2965067394.py:17: DeprecationWarning: In future, it will be an error for 'np.bool' scalars to be interpreted as an index
  x = torch.FloatTensor([self.x1[index], self.x2[index]])
/tmp/ipython-input-2965067394.py:18: DeprecationWarning: In future, it will be an error for 'np.bool' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])


Epoch : 1000, Cost : 0.692
Epoch : 2000, Cost : 0.692
Epoch : 3000, Cost : 0.692
Epoch : 4000, Cost : 0.692
Epoch : 5000, Cost : 0.692
Epoch : 6000, Cost : 0.692
Epoch : 7000, Cost : 0.692
Epoch : 8000, Cost : 0.692
Epoch : 9000, Cost : 0.692
Epoch : 10000, Cost : 0.692


In [9]:
# 모델 평가
with torch.no_grad():
  model.eval()
  inputs = torch.FloatTensor([
      [0, 0],
      [0, 1],
      [1, 0],
      [1, 1]
  ]).to(device)
  outputs = model(inputs)

  print("---------")
  print(outputs)
  print(outputs <= 0.5)

---------
tensor([[0.4673],
        [0.4997],
        [0.5027],
        [0.5351]], device='cuda:0')
tensor([[ True],
        [ True],
        [False],
        [False]], device='cuda:0')


cost가 0.692로 일정하게 나오는데, 단층 퍼셉트론으로는 XOR 문제를 해결할 수 없기 때문이다.  
또한, 모델에 input을 넣었을 때도 0.5 근처로 출력돼 학습이 제대로 이루어지지 않은 것을 확인할 수 있다.  
<br>
따라서 레이어를 하나 추가한 다층 퍼셉트론 구조로 변경해보겠다.

# 다층 퍼셉트론

In [17]:
# 커스텀 모델(레이어 하나 추가)
class CustomModel(nn.Module):
  def __init__(self):
    super().__init__()

    self.layer1 = nn.Sequential(
        nn.Linear(2, 2),
        nn.Sigmoid()
    )
    self.layer2 = nn.Sequential(
        nn.Linear(2, 1),
        nn.Sigmoid()
    )

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    return x

In [18]:
# model, Loss, optimizer
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [19]:
# 모델 학습
for epoch in range(10000):
  cost = 0.0

  for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)
    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    cost += loss

  cost = cost / len(train_dataloader)

  if (epoch + 1) % 1000 == 0:
    print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

/tmp/ipython-input-2965067394.py:17: DeprecationWarning: In future, it will be an error for 'np.bool' scalars to be interpreted as an index
  x = torch.FloatTensor([self.x1[index], self.x2[index]])
/tmp/ipython-input-2965067394.py:18: DeprecationWarning: In future, it will be an error for 'np.bool' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])


Epoch : 1000, Cost : 0.677
Epoch : 2000, Cost : 0.464
Epoch : 3000, Cost : 0.114
Epoch : 4000, Cost : 0.047
Epoch : 5000, Cost : 0.028
Epoch : 6000, Cost : 0.020
Epoch : 7000, Cost : 0.015
Epoch : 8000, Cost : 0.012
Epoch : 9000, Cost : 0.010
Epoch : 10000, Cost : 0.009


In [20]:
# 모델 평가
with torch.no_grad():
  model.eval()
  inputs = torch.FloatTensor([
      [0, 0],
      [0, 1],
      [1, 0],
      [1, 1]
  ]).to(device)
  outputs = model(inputs)

  print("---------")
  print(outputs)
  print(outputs <= 0.5)

---------
tensor([[0.0102],
        [0.9919],
        [0.9919],
        [0.0093]], device='cuda:0')
tensor([[ True],
        [False],
        [False],
        [ True]], device='cuda:0')


레이어를 하나 더 추가하니 cost가 유의미하게 감소하고, XOR 문제도 잘 해결했다.